In [ ]:
# Importing libraries and modules

import geopandas as gpd
from geopandas.tools import sjoin
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from shapely import wkt
from scipy.spatial import cKDTree
from shapely.geometry import Point
from shapely.ops import nearest_points
import geojson

In [ ]:
# Reading parking spaces geojson file

#df_parking = gpd.read_file("F:/Berlin-DOP_2020/Processed images_new/Final.gpkg", crs='EPSG:25833', layer = "Pred_distance_calculated")
df_parking = gpd.read_file("F:/Berlin-DOP_2020/Processed images_new/Predictions.geojson", crs='EPSG:25833')

In [ ]:
df_parking

In [ ]:
# Adding the polygon geometries to a seperate column
df_parking['parking_polygon'] = df_parking['geometry']

# Adding the area of parking to a seperate column
df_parking["parking_area_m2"] = df_parking['geometry'].area

# Setting the center point of each parking as the geometry column
df_parking.geometry = df_parking.representative_point()

# Converting the parking polygons to objects
df_parking['parking_polygon'] = df_parking['parking_polygon'].astype(str)

In [ ]:
df_parking

In [ ]:
# Copying the parking dataframe to merge later
df_parking_copy = df_parking.copy()

# Adding two new columns to add nearest parking space geometry
#df_parking.insert(5, 'nearest_geometry_index', None)
df_parking.insert(5, 'nearest_geometry', None)

In [ ]:
# Finding the nearest parking space
for index, row in df_parking.iterrows():
    point = row.geometry
    multipoint = df_parking.drop(index, axis=0).geometry.unary_union
    try:
        queried_geom, nearest_geom = nearest_points(point, multipoint)
        df_parking.loc[index, 'nearest_geometry'] = nearest_geom
    except:
        pass

In [ ]:
# Setting the nerest geometry ad dataframe's geometry
df_parking = df_parking.set_geometry("nearest_geometry")
df_parking['geometry'] = df_parking['geometry'].astype(str)

In [ ]:
df_parking

In [ ]:
# Calculating the distance from each parking space to the nearest parking space
df_parking["nearest_geometry_dis"] = df_parking.distance(df_parking_copy)

In [ ]:
# Adding the other data columns
df_merge = df_parking_copy.loc[:, ["id","geometry","parking_area_m2"]]

In [ ]:
df_merge = df_merge.rename(columns={'geometry': 'nearest_geometry', 'parking_area_m2': 'parking_area_nearest_m2'})

In [ ]:
# Joining the final dataframes
Merged_df = df_parking.merge(df_merge, on='nearest_geometry', how='inner')

In [ ]:
Merged_df

In [ ]:
# Setting the geometry to the parking space polygons
Merged_df['geometry'] = Merged_df['geometry'].astype(str)
Merged_df['nearest_geometry'] = Merged_df['nearest_geometry'].astype(str)

# Converting to a geo dataframe
Merged_df['parking_polygon'] = Merged_df['parking_polygon'].apply(wkt.loads)
Merged_df = gpd.GeoDataFrame(Merged_df, geometry="parking_polygon", crs='EPSG:25833')

In [ ]:
Merged_df.dtypes

In [ ]:
Merged_df.to_file("F:/Berlin-DOP_2020/Processed images_new/Final.gpkg", crs='EPSG:25833', driver="GPKG", Layer= "Predict_with_info")
#Merged_df.to_file("F:/Berlin-DOP_2020/Processed images_new/parking_dist.gpkg", driver="GPKG", layer="parking_dist")

In [ ]:
Merged_df

In [ ]:
# def ckdnearest(gdA, gdB):
#     assert len(gdA) > 0
#     assert len(gdB) > 0
#     nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
#     nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
#     btree = cKDTree(nB)
#     dist, idx = btree.query(nA, k=1)
#     gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
#     gdf = pd.concat(
#         [gdA.reset_index(drop=True), gdB_nearest, pd.Series(dist, name="dist")],
#         axis=1,
#     )

#     return gdf

In [ ]:
# def catcherror(x):
#     try:
#         return x.x
#     except:
#         return np.nan

In [ ]:
# df_parking.loc[df_parking.geometry.apply(lambda x: catcherror(x)).isna(),:]

In [ ]:
# ckdnearest(df_parking, df_parking_copy)

In [ ]:
# from scipy import spatial

# # get list of points
# points = df_parking['geometry'].apply(
#                 lambda g:[g.x,g.y]).tolist()
# #spatially organising the points on a tree for quick nearest neighbors calc
# kdtree = spatial.KDTree(points)

# #calculates the nearest neighbors of each point
# _ , neighs = kdtree.query(points, k=2)

# # remove itself as neighbor
# neighs = neighs[:,1]


# # add column to df
# df_parking['nearest_points'] = df_parking.iloc[neighs, 3].tolist()